Ratatui is a crate for building terminal user interfaces in Rust.

In this post, we'll discuss the `Rect` and `Layout` primitives of `ratatui`.

In [14]:
//| code-fold: true
:dep ratatui = "0.26.2"
:dep ratatui-macros = "0.4.0"
    
fn show_html<D>(content: D) where D: std::fmt::Display {
    println!(r#"EVCXR_BEGIN_CONTENT text/html
<div style="display: flex; justify-content:start; gap: 1em; margin: 1em">
{}
</div>
EVCXR_END_CONTENT"#, content);
}


## Layout primitives

We already saw that `Rect` is one of the primitives for rendering a widget.

We can create a `Rect` using `Rect::new(x, y, width, height)`:

In [7]:
use ratatui::widgets::Widget;

let (x, y, width, height) = (0, 0, 50, 5); 
let area = ratatui::layout::Rect::new(x, y, width, height);
let mut buf = ratatui::buffer::Buffer::empty(area);
ratatui::widgets::Block::bordered().render(area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌────────────────────────────────────────────────┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

You can create inner `Rect`s by using the `Rect::inner` method:

In [10]:
let mut buf = ratatui::buffer::Buffer::empty(area);

let (horizontal, vertical) = (5, 1);
let inner_area = area.inner(&ratatui::layout::Margin::new(horizontal, vertical));

ratatui::widgets::Block::bordered().render(inner_area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "                                                  ",
        "     ┌──────────────────────────────────────┐     ",
        "     │                                      │     ",
        "     └──────────────────────────────────────┘     ",
        "                                                  ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Ratatui also has a layout solver using [the cassowary algorithm](https://en.wikipedia.org/wiki/Cassowary_(software)?useskin=vector).



In [11]:
use ratatui::layout::{Layout, Constraint};

let [first, second] = ratatui::layout::Layout::horizontal([Constraint::Length(10), Constraint::Length(10)]).areas(area);

let mut buf = ratatui::buffer::Buffer::empty(area);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌first───┐┌second──┐                              ",
        "│        ││        │                              ",
        "│        ││        │                              ",
        "│        ││        │                              ",
        "└────────┘└────────┘                              ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

`ratatui-macros` has a couple of macros to make some of this boilerplate simpler.

In [15]:
let [first, second] = ratatui::layout::Layout::horizontal([Constraint::Length(10), Constraint::Length(10)]).areas(area);
// OR
let [first, second] = ratatui_macros::horizontal![==10, ==10].areas(area);

In [13]:
use ratatui_macros::{horizontal, vertical};

let (x, y, width, height) = (0, 0, 50, 6); 
let area = ratatui::layout::Rect::new(x, y, width, height);

let mut buf = ratatui::buffer::Buffer::empty(area);

let [top, middle, bottom] = vertical![*=1, *=1, *=1].areas(area);

let [first, second] = horizontal![==10, ==10].areas(top);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

let [first, second] = horizontal![==10, ==10].flex(ratatui::layout::Flex::Center).areas(middle);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

let [first, second] = horizontal![==10, ==10].flex(ratatui::layout::Flex::End).areas(bottom);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 6 },
    content: [
        "┌first───┐┌second──┐                              ",
        "└────────┘└────────┘                              ",
        "               ┌first───┐┌second──┐               ",
        "               └────────┘└────────┘               ",
        "                              ┌first───┐┌second──┐",
        "                              └────────┘└────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

## Conclusion

In the next post, we'll examine how text primitives work.